# GeoEnrichment

GeoEnrichment provides the ability to 
* get facts about a location or area. 
* information about the people, places, and businesses 
 * in a specific area or 
 * within a certain distance or drive time from a location.
* large collection of data sets including population, income, housing, consumer behavior, and the natural environment.
* Site analysis is a popular application

In [ ]:
from arcgis.gis import GIS
from arcgis.geoenrichment import *

gis = GIS('https://www.arcgis.com', 'arcgis_python')

## GeoEnrichment coverage

In [ ]:
countries = get_countries()
print("Number of countries for which GeoEnrichment data is available: " + str(len(countries)))

#print a few countries for a sample
countries[0:10]

### Filtering countries by properties

In [ ]:
[c.properties.name for c in countries if c.properties.continent == 'Australia']

## Discovering information for a country
* Data collections, 
* Sub-geographies and 
* Available reports for a country

In [ ]:
Country.

In [ ]:
usa = Country.get('US')

Commonly used properties for the country are accessible using `Country.properties`.

In [ ]:
usa.properties.name

### Data collections and analysis variables

In [ ]:
df = usa.data_collections

# print a few rows of the DataFrame
df.head()

In [ ]:
# call the shape property to get the total number of rows and columns
df.shape

In [ ]:
# get all the unique data collections available
len(df.index.unique())

Query the `Age` data collection and get all the unique `analysisVariable`s under that collection

In [ ]:
df.loc['Age']['analysisVariable'].unique()

In [ ]:
# view a sample of the `Age` data collection
df.loc['Age'].head()

### Enriching an address

In [ ]:
enrich(study_areas=["380 New York St Redlands CA 92373"],  
       data_collections=['Age'])

# Reports

In [20]:
# print a sample of the reports available for USA
usa.reports.head(10)

,id,title,categories,formats
0,census2010_profile,2010 Census Profile,[Demographics],"[pdf, xlsx]"
1,acs_housing,ACS Housing Summary,[Demographics],"[pdf, xlsx]"
2,acs_population,ACS Population Summary,[Demographics],"[pdf, xlsx]"
3,55plus,Age 50+ Profile,[Demographics],"[pdf, xlsx]"
4,agesexrace,Age by Sex by Race Profile,[Demographics],"[pdf, xlsx]"
5,agesex,Age by Sex Profile,[Demographics],"[pdf, xlsx]"
6,cex_auto,Automotive Aftermarket Expenditures,[Consumer Spending],"[pdf, xlsx]"
7,business_loc,Business Locator,[Business],"[pdf, xlsx]"
8,business_summary,Business Summary,[Business],"[pdf, xlsx]"
9,community_profile,Community Profile,[Demographics],"[pdf, xlsx]"


In [21]:
# total number of reports available
usa.reports.shape

(49, 4)

### Creating Reports

In [22]:
report = create_report(study_areas=["380 New York Street, Redlands, CA"],
                     report="tapestry_profileNEW",
                     export_format="PDF", 
                     out_folder=r"c:\temp", out_name="esri_tapestry_profile.pdf")
report

'c:\\temp\\esri_tapestry_profile.pdf'

## Finding named statistical areas

Each country has several named statistical areas in a hierarchy of geography levels (such as states, counties, zip codes, etc).

In [23]:
%config IPCompleter.greedy=True

In [ ]:
usa.subgeographies.states['Delaware'].counties['New_Castle_County'].tracts[]

In [ ]:
usa.subgeographies.states['California'].counties['San_Bernardino_County']

In [ ]:
usa.subgeographies.states['California'].counties['San_Bernardino_County'].tracts['060710001.03']

In [ ]:
usa.subgeographies.states['California'].zip5['92373']

The named areas can also be drawn on a map, as they include a `geometry` property.

In [ ]:
m = gis.map('Redlands, CA', zoomlevel=11)
m

In [ ]:
m.draw(usa.subgeographies.states['California'].zip5['92373'].geometry)

# Different geography levels for different country

In [ ]:
india = Country.get('India')

In [ ]:
india.subgeographies.states['Uttar_Pradesh'].districts['Baghpat'].subdistricts['Baraut']

### Searching for named areas within a country

In [25]:
riversides_in_usa = usa.search('Riverside')
print("number of riversides in the US: " + str(len(riversides_in_usa)))

# list a few of them
riversides_in_usa[:10]

number of riversides in the US: 83


[<NamedArea name:"Riverside" area_id="147435", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147436", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147437", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147438", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147439", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147440", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147441", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147442", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147443", level="Cities", country="United States">,
 <NamedArea name:"Riverside" area_id="147444", level="Cities", country="United States">]

For instance, you can make a map of all the riversides in the US

In [26]:
usamap = gis.map('United States', zoomlevel=4)
usamap

Widget Javascript not detected.  It may not be installed or enabled properly.


In [27]:
for riverside in riversides_in_usa:
    usamap.draw(riverside.geometry)

#### Filtering named areas by geography level

In [ ]:
[level['id'] for level in usa.levels]

In [ ]:
usa.search(query='Riverside', layers=['US.Counties'])

## Finding businesses

In [ ]:
businesses = find_businesses(search_string="Starbucks",
                          return_geometry=True,
                          spatial_filter={"Locations":["NY,TONAWANDA,14150",
                                                       "NJ, CAMDEN, 08102",
                                                       "KY,LOUISVILLE,40204",
                                                       "WA,SEATTLE,98108"]})
# print a sample
businesses

## Study Areas

### Accepted forms of study areas

- **Street address locations** - Locations can be passed as strings of input street addresses, points of interest or place names.
    + **Example:** `"380 New York St, Redlands, CA"`

- **Multiple field input addresses** - Locations described as multiple field input addresses, using dictionaries.
    + **Example:** 
        {"Address" : "380 New York Street",
        "City" : "Redlands",
        "Region" : "CA",
        "Postal" : 92373}    
 
- **Point and line geometries** - Point and line locations, using `arcgis.geometry` instances.
    + **Example Point Location: ** 
    
    `arcgis.geometry.Geometry({"x":-122.435,"y":37.785})`
    
    + ** Example Point location obtained using find_businesses() above: **
     
     `arcgis.geometry.Geometry(businesses.iloc[0]['SHAPE'])`

- **Buffered study areas** - `BufferStudyArea` instances to change the ring buffer size or create drive-time service areas around points specified using one of the above methods. BufferStudyArea allows you to buffer point and street address study areas. They can be created using the following parameters:
        * area: the point geometry or street address (string) study area to be buffered
        * radii: list of distances by which to buffer the study area, eg. [1, 2, 3]
        * units: distance unit, eg. Miles, Kilometers, Minutes (when using drive times/travel_mode)
        * overlap: boolean, uses overlapping rings/network service areas when True, or non-overlapping disks when False
        * travel_mode: None or string, one of the supported travel modes when using network service areas
    + **Example Buffered Location: ** 
    
    `pt = arcgis.geometry.Geometry({"x":-122.435,"y":37.785})
    buffered_area = BufferStudyArea(area=pt, radii=[1,2,3], units="Miles", overlap=False)` 

- **Network service areas** - `BufferStudyArea` also allows you to define drive time service areas around points as well as other advanced service areas such as walking and trucking.
    + **Example: **
    
    `pt = arcgis.geometry.Geometry({"x":-122.435,"y":37.785})
    buffered_area = BufferStudyArea(area=pt, radii=[1,2,3], units="Minutes", travel_mode="Driving")` 

- **Named statistical areas** - 
    + **Example:** 
    
    `usa.subgeographies.states['California'].zip5['92373']`
   
- **Polygon geometries** - Locations can given as polygon geometries.
    + **Example Polygon geometry: ** 
    
    `arcgis.geometry.Geometry({"rings":[[[-117.185412,34.063170],[-122.81,37.81],[-117.200570,34.057196],[-117.185412,34.063170]]],"spatialReference":{"wkid":4326}})`


### Example: Enriching a named statistical area
Enriching zip code 92373 in California using the 'Age' data collection:

In [ ]:
redlands = usa.subgeographies.states['California'].zip5['92373']

In [ ]:
enrich(study_areas=[redlands], data_collections=['Age'] )

### Example: Enrich all counties in a state

In [ ]:
ca_counties = usa.subgeographies.states['California'].counties

In [ ]:
counties_df = enrich(study_areas=ca_counties, data_collections=['Age'])
counties_df.head(10)

In [ ]:
m = gis.map('California')
m

In [ ]:
lyr = gis.content.import_data(df=counties_df, title="CA county population")

In [ ]:
m.add_layer(lyr.layers[0], {'renderer': 'ClassedColorRenderer', 
                            'field_name':'FEM0'})

### Example: Using comparison levels

In [ ]:
enrich(study_areas=[redlands], data_collections=['Age'], 
       comparison_levels=['Counties', 'States'])

### Example: Buffering locations using non overlapping disks 

The example below creates non-overlapping disks of radii 1, 3 and 5 Miles respectively from a street address and enriches these using the 'Age' data collection.

In [ ]:
buffered = BufferStudyArea(area='380 New York St Redlands CA 92373',
                           radii=[1,3,5], units='Miles', overlap=False)
enrich(study_areas=[buffered], data_collections=['Age'])

### Example: Using drive times as study areas
    
The example below creates 5 and 10 minute drive times from a street address and enriches these using the 'Age' data collection.

In [ ]:
buffered = BufferStudyArea(area='380 New York St Redlands CA 92373', 
                           radii=[5, 10], units='Minutes', 
                           travel_mode='Driving')
drive_time_df = enrich(study_areas=[buffered], data_collections=['Age'])

In [ ]:
drive_time_df

### Visualize results on a map

The returned spatial dataframe can be visualized on a map as shown below:

In [ ]:
redlands_map = gis.map('Redlands, CA')
redlands_map.basemap = 'dark-gray-vector'
redlands_map

In [ ]:
redlands_map.draw(drive_time_df.to_featureset())

## Saving GeoEnrichment Results

In [ ]:
item = gis.content.import_data(df=drive_time_df, title="Age statistics within 5,10 minutes of drive time from Esri")

In [ ]:
item

In [ ]:
item.delete()